We need to fix the waveform generation so that it isn't just 1 or 0 for a pixel. We need the waveform to be set up for temporal accuracy

In [3]:
from PIL import Image
import os
import numpy as np

In [42]:
# For each pixel we need 100 points, not 1. If it is a 1, we append 50 1's and 50 0's. For 0, 100 0's.
n = 100
black = np.zeros(n)
white = np.zeros(n)
white[: n // 2] = 1

# Rewriting function to add white or black for each pixel
def create_correct_waveform(image_path, serpentine = True):
    waveforms = []
    img = Image.open(image_path)
    binary = (np.array(img) > 128).astype(np.uint8)
    # appends each row to the list
    for row_index in range(binary.shape[0]):
        row = binary[row_index]
        for column in row:
            if column == 0:
                waveforms.append(black)
            else:
                waveforms.append(white)
    return np.concatenate(waveforms)
correct_waveform = create_correct_waveform(image_path)
correct_waveform

array([0., 0., 0., ..., 0., 0., 0.], shape=(100000000,))

In [ ]:
def create_bidirectional_waveforms(image_dir, serpentine=True):
    waveforms = []

    image_files = sorted([
        f for f in os.listdir(image_dir)
        if f.endswith(".png")
    ])

    for i, filename in enumerate(image_files):
        path = os.path.join(image_dir, filename)
        img = Image.open(path).convert("L")
        binary = (np.array(img) > 128).astype(np.uint8)
        for row_index in range(binary.shape[0]):
            row = binary[row_index]
            for column in row:
                if column == 0:
                    waveforms.append(black)
                else:
                    waveforms.append(white)
        return np.concatenate(waveforms)

In [ ]:
# Need to save as a binary waveform now
waveform_normalized = correct_waveform.astype(np.float64) * 2 -1
waveform_normalized.astype('<f8').tofile(f"correct_waveform.bin")

Now each pixel has the correct timing. However, it isn't synchronized for the scanning. We need to have it synchronized to the mirror scanner.

This means that each row needs a gap.

In [45]:
import nifgen
import time

with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = 100_000_000
    wf_handle = session.create_waveform(waveform_normalized)
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.trigger_mode = nifgen.TriggerMode.CONTINUOUS
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.configure_arb_waveform(wf_handle, gain=1, offset=0)
    session.initiate()
    time.sleep(5)